# Main explanation

#### Good lecture : https://www.cambridge.org/core/journals/journal-of-materials-research/article/double-perovskite-sr2bbo6-oxides-for-hightemperature-thermoelectric-power-generationa-review/87CFBEC724851ABAFEDB893AD7E747E8/core-reader

#### ISCD QUERY
One search on the ICSD was made for spacegroupe : Fm-3m with number of elements <=2 - 4776 queries
The second was made for spacegroupe : Fm-3m with number of elements >2 - 6123 queries
The third was made for spacegroupe : Fm-3m with number of elements >3 - 2037 queries

## Importation

In [79]:
import pandas as pd
import numpy as np
import time  
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as pltimport 
import sys
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

## Openning from text file

In [506]:
df = pd.DataFrame(pd.read_table("ICSD-halides.txt")) #even thoug the data is already frammed
#df1 = pd.read_json("ISCD3-2/ISCD3_CollCode28065.cif")
#df.to_csv('ISCD333.csv')

In [450]:
df.columns

Index(['StructuredFormula', 'StructureType', 'SumFormula', 'ChemicalName',
       'Unnamed: 4'],
      dtype='object')

In [507]:
df.rename(columns = {"Unnamed: 4": "Perovskite_label"},  
          inplace = True)
#df = df.rename(columns = lambda x: x+':')
df['Perovskite_label']=0
df.fillna('-',inplace = True)
df

,StructuredFormula,StructureType,SumFormula,ChemicalName,Perovskite_label
0,Cs2 Na (Cr F6),Ba2NiTeO6,Cr1 Cs2 F6 Na1,Dicesium sodium hexafluorochromate(III),0
1,Cs2 Na (Fe F6),Ba2NiTeO6,Cs2 F6 Fe1 Na1,Dicesium sodium hexafluoroferrate(III),0
2,Cs2 Li (Ga F6),-,Cs2 F6 Ga1 Li1,Dicesium lithium hexafluorogallate,0
3,(Se (Se)3 Te2) (As F6)2,Te2Se4(SbF6)2,As2 F12 Se4 Te2,Mue-seleno-mue-triseleno-ditellurium bis(hexaf...,0
4,Cs2 Na Bi Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Bi1 Cl6 Cs2 Na1,Dicesium sodium bismuth chloride,0
...,...,...,...,...,...
191,Cs2 Na (Y0.75 Ho0.25) Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Cl6 Cs2 Ho0.25 Na1 Y0.75,Cesium sodium yttrium holmium chloride (2/1/0....,0
192,Cs2 Na (Y0.50 Ho0.50) Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Cl6 Cs2 Ho0.5 Na1 Y0.5,Cesium sodium yttrium holmium chloride (2/1/0....,0
193,Cs2 Na (Y0.25 Ho0.75) Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Cl6 Cs2 Ho0.75 Na1 Y0.25,Cesium sodium yttrium holmium chloride (2/1/0....,0
194,Cs2 Na Ho Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Cl6 Cs2 Ho1 Na1,Cesium sodium holmium chloride (2/1/1/6),0


In [455]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   StructuredFormula  196 non-null    object
 1   StructureType      196 non-null    object
 2   SumFormula         196 non-null    object
 3   ChemicalName       196 non-null    object
 4   Perovskite_label   196 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.8+ KB


In [ ]:
# get rid of duplicate and polymorpishm and get rid of complex so stoch. only integer

### The one that are perovskite

In [508]:
for l in range(len(df)):
    if 'Perovskite' in df.StructureType[l]:
        df.Perovskite_label[l] = 1
        print(df.index[l])
        print(df.StructureType[l])
    if "perovskite" in df.StructureType:
        df.Perovskite_label = 1
        print(df.index[l])

51
Perovskite#NaBaLiNiF6
52
Perovskite#Na3AlF6
62
Perovskite#Ba2LaRuO6#Cu2NdNbO6
79
Perovskite#Na3AlF6
154
Perovskite#K2LiAlF6


In [509]:
for l in range(len(df)):
    if df.Perovskite_label[l] == 1:
             print (df.SumFormula[l])

Ba1 F6 Li1 Na1 Ni1
F6 K1 Rb2 Y1
Al1 F6 Li1 Na2
Al1 F6 Li1 Na2
Al1 F6 K2 Li1


### Get rid of COMPLEX

In [511]:
a = []

for l in range(len(df)):
    if '.' in df.StructuredFormula[l]:
        #print (df.StructuredFormula[l])
        a.append(l)
        
df_complex = df.drop(a, axis=0)
df_complex.sort_values(by=['Perovskite_label'], ascending=False, inplace=True)
df_complex.reset_index(drop=True, inplace=True)
df_complex

,StructuredFormula,StructureType,SumFormula,ChemicalName,Perovskite_label
0,Na Ba Li Ni F6,Perovskite#NaBaLiNiF6,Ba1 F6 Li1 Na1 Ni1,Sodium barium lithium nickel fluoride,1
1,Na2 Li (Al F6),Perovskite#Ba2LaRuO6#Cu2NdNbO6,Al1 F6 Li1 Na2,Disodium lithium hexafluoroaluminate,1
2,Li Na2 (Al F6),Perovskite#Na3AlF6,Al1 F6 Li1 Na2,Lithium disodium hexafluoroaluminateide,1
3,K2 Li (Al F6),Perovskite#K2LiAlF6,Al1 F6 K2 Li1,Dipotassium lithium hexafluoroaluminate,1
4,Rb2 K Y F6,Perovskite#Na3AlF6,F6 K1 Rb2 Y1,Dirubidium potassium yttrium fluoride,1
...,...,...,...,...,...
182,Rb2 K In F6,Elpasolite#K2NaAlF6#Ba2CaWO6,F6 In1 K1 Rb2,Dirubidium potassium indium hexafluoride,0
183,Cs K2 Bi Cl6,Rb3YCl6,Bi1 Cl6 Cs1 K2,Cesium dipotassium bismuth hexachloride,0
184,Cs2 Ag Bi Br6,Elpasolite#K2NaAlF6#Ba2CaWO6,Ag1 Bi1 Br6 Cs2,Dicesium silver bismuth hexabromide,0
185,Cs2 Bi Ag Cl6,Elpasolite#K2NaAlF6#Ba2CaWO6,Ag1 Bi1 Cl6 Cs2,Dicesium bismuth silver hexachloride,0


### Get rid of DUPLICATE taking into account polymorphism

In [515]:
seen = []
uniq = []
for l in range(len(df_complex)):
    uniq.append(l)
    if df_complex.StructuredFormula[l] not in seen:
    #print(df_complex.StructuredFormula[l])
        del uniq[-1]
        seen.append(df_complex.StructuredFormula[l])

df_complex_multiplicate = df_complex.drop(uniq, axis=0)
df_complex_multiplicate.reset_index(drop=True, inplace=True)
df_complex_multiplicate

,StructuredFormula,StructureType,SumFormula,ChemicalName,Perovskite_label
0,Na Ba Li Ni F6,Perovskite#NaBaLiNiF6,Ba1 F6 Li1 Na1 Ni1,Sodium barium lithium nickel fluoride,1
1,Na2 Li (Al F6),Perovskite#Ba2LaRuO6#Cu2NdNbO6,Al1 F6 Li1 Na2,Disodium lithium hexafluoroaluminate,1
2,Li Na2 (Al F6),Perovskite#Na3AlF6,Al1 F6 Li1 Na2,Lithium disodium hexafluoroaluminateide,1
3,K2 Li (Al F6),Perovskite#K2LiAlF6,Al1 F6 K2 Li1,Dipotassium lithium hexafluoroaluminate,1
4,Rb2 K Y F6,Perovskite#Na3AlF6,F6 K1 Rb2 Y1,Dirubidium potassium yttrium fluoride,1
...,...,...,...,...,...
138,Cs2 Au2 Br6,CsAuCl3,Au2 Br6 Cs2,Dicesium dibromoaurate(I) tetrabromoaurate(III),0
139,Au2 I6 Rb2,-,Au2 I6 Rb2,Dirubidium diiodoaurate(I) tetraiodoaurate(III),0
140,Cs2 Au2 I6,CsAuCl3,Au2 Cs2 I6,"Dicesium digold(I,III) hexaiodide",0
141,Cs K2 Bi Cl6,Rb3YCl6,Bi1 Cl6 Cs1 K2,Cesium dipotassium bismuth hexachloride,0


## Code - FINISHED and now extraction

In [525]:
df_complex_multiplicate.to_csv('Parsed-halides.csv', columns = ['StructuredFormula', 'Perovskite_label'], index=False)

## DataFrame for the Oxide Perovskite

In [576]:
dfo1 = pd.DataFrame(pd.read_table("ICSD-oxides-st.txt")) #even thoug the data is already frammed
dfo2 = pd.DataFrame(pd.read_table("ICSD-oxides-2nd.txt"))
dfo3 = pd.DataFrame(pd.read_table("ICSD-oxides-3nd.txt"))
#df1 = pd.read_json("ISCD3-2/ISCD3_CollCode28065.cif")
#df.to_csv('ISCD333.csv')


#dfoxide = pd.concat([dfo1, dfo2, dfo3], sort=False)
dfoxide = pd.concat([dfo1, dfo2,dfo3])
dfoxide

,CollectionCode,HMS,StructuredFormula,StructureType,Pressure,SumFormula,ChemicalName,Journal,PublicationYear,Unnamed: 9
0,395,P n 1 1,Rb (B2 Nb O6),NaN,0.101325,B2 Nb1 O6 Rb1,Rubidium diboroniobate,"Acta Crystallographica, Section B: Structural ...",1976,NaN
1,1862,P b c a,Zn Mg (Si2 O6),Pyroxene#MgSiO3,0.101325,Mg1 O6 Si2 Zn1,Zinc magnesium disilicate,"Acta Crystallographica, Section B: Structural ...",1975,NaN
2,2082,P n 21 a,Tl Nb B2 O6,NaN,0.101325,B2 Nb1 O6 Tl1,Thallium niobium borate *,"Acta Crystallographica, Section B: Structural ...",1974,NaN
3,2135,C 1 2/c 1,Na In Si2 O6,Pyroxene#CaMg(SiO3)2,0.101325,In1 Na1 O6 Si2,Sodium indium silicate *,"Acta Crystallographica, Section B: Structural ...",1974,NaN
4,2199,C 1 2/c 1,Na K (V2 O6),Pyroxene#CaMg(SiO3)2,0.101325,K1 Na1 O6 V2,Sodium potassium catena-divanadate,Journal of Solid State Chemistry,1978,NaN
...,...,...,...,...,...,...,...,...,...,...
429,415471,P 1 21/n 1,Ag2 (Te2 O6),NaN,0.101325,Ag2 O6 Te2,Disilver telluro(IV)tellurate - I,Zeitschrift fuer Anorganische und Allgemeine C...,2005,NaN
430,425416,P 63 m c,Rb In (H P O3)2,KV(HPO3)2,0.101325,H2 In1 O6 P2 Rb1,Rubidium indium bis(phosphate(III)),CrystEngComm,2013,NaN
431,425417,R -3 m H,Cs In (H P O3)2,NaN,0.101325,H2 Cs1 In1 O6 P2,Cesium indium bis(phosphate(III)),CrystEngComm,2013,NaN
432,429209,I m m m,Cs2 U Si O6,NaN,0.101325,Cs2 O6 Si1 U1,Dicesium uranium(VI) silicate,Inorganic Chemistry,2015,NaN


In [575]:
len(dfoxide)

2434

In [580]:
dfoxide.rename(columns = {"Unnamed: 9": "Perovskite_label"},  
          inplace = True)
#df = df.rename(columns = lambda x: x+':')
dfoxide['Perovskite_label']=0
dfoxide.fillna('-',inplace = True)
dfoxide.reset_index(drop=True, inplace=True)
dfoxide

,CollectionCode,HMS,StructuredFormula,StructureType,Pressure,SumFormula,ChemicalName,Journal,PublicationYear,Perovskite_label
0,395,P n 1 1,Rb (B2 Nb O6),-,0.101325,B2 Nb1 O6 Rb1,Rubidium diboroniobate,"Acta Crystallographica, Section B: Structural ...",1976,0
1,1862,P b c a,Zn Mg (Si2 O6),Pyroxene#MgSiO3,0.101325,Mg1 O6 Si2 Zn1,Zinc magnesium disilicate,"Acta Crystallographica, Section B: Structural ...",1975,0
2,2082,P n 21 a,Tl Nb B2 O6,-,0.101325,B2 Nb1 O6 Tl1,Thallium niobium borate *,"Acta Crystallographica, Section B: Structural ...",1974,0
3,2135,C 1 2/c 1,Na In Si2 O6,Pyroxene#CaMg(SiO3)2,0.101325,In1 Na1 O6 Si2,Sodium indium silicate *,"Acta Crystallographica, Section B: Structural ...",1974,0
4,2199,C 1 2/c 1,Na K (V2 O6),Pyroxene#CaMg(SiO3)2,0.101325,K1 Na1 O6 V2,Sodium potassium catena-divanadate,Journal of Solid State Chemistry,1978,0
...,...,...,...,...,...,...,...,...,...,...
2429,415471,P 1 21/n 1,Ag2 (Te2 O6),-,0.101325,Ag2 O6 Te2,Disilver telluro(IV)tellurate - I,Zeitschrift fuer Anorganische und Allgemeine C...,2005,0
2430,425416,P 63 m c,Rb In (H P O3)2,KV(HPO3)2,0.101325,H2 In1 O6 P2 Rb1,Rubidium indium bis(phosphate(III)),CrystEngComm,2013,0
2431,425417,R -3 m H,Cs In (H P O3)2,-,0.101325,H2 Cs1 In1 O6 P2,Cesium indium bis(phosphate(III)),CrystEngComm,2013,0
2432,429209,I m m m,Cs2 U Si O6,-,0.101325,Cs2 O6 Si1 U1,Dicesium uranium(VI) silicate,Inorganic Chemistry,2015,0


In [ ]:
# get rid of duplicate and polymorpishm and get rid of complex so stoch. only integer

## Labelling

In [581]:
for l in range(len(dfoxide)):
    if 'Perovskite' in dfoxide.StructureType[l]:
        dfoxide.Perovskite_label[l] = 1
        print(dfoxide.index[l])
        print(dfoxide.StructureType[l])
    if "perovskite" in dfoxide.StructureType:
        dfoxide.Perovskite_label = 1
        print(dfoxide.index[l])

6
Perovskite#Ba2LaRuO6#Cu2NdNbO6
7
Perovskite#Ba2LaRuO6#Cu2NdNbO6
8
Perovskite#Ba2LaRuO6(I1-)
9
Perovskite#Ba2LaRuO6#Cu2NdNbO6
32
Perovskite#Ba2LaRuO6#Cu2NdNbO6
33
Perovskite#Na3AlF6
34
Perovskite#Na3AlF6
98
Perovskite#K2LiAlF6
100
Perovskite#Ba2LaRuO6#Cu2NdNbO6
101
Perovskite#Ba2LaRuO6(I1-)
104
Perovskite#Ba2LaRuO6#Cu2NdNbO6
105
Perovskite_double#PrBaMn2O5+x
109
Perovskite#Sr3CoSb2O9
117
Perovskite#Ba2LaRuO6#Cu2NdNbO6
121
Perovskite#Ba2LaRuO6#Cu2NdNbO6
124
Perovskite#Ba2LaRuO6#Cu2NdNbO6
142
Perovskite#Ba2LaRuO6#Cu2NdNbO6
143
Perovskite#Ba2LaRuO6#Cu2NdNbO6
144
Perovskite#Ba2LaRuO6#Cu2NdNbO6
172
Perovskite#Na3AlF6
195
Perovskite#Sr3CoSb2O9
201
Perovskite#BaBiO3(mS20)
212
Perovskite#Na3AlF6
234
Perovskite#Ba2LaRuO6#Cu2NdNbO6
238
Perovskite#Ba2LaRuO6#Cu2NdNbO6
240
Perovskite#Ba2LaRuO6#Cu2NdNbO6
241
Perovskite#BaBiO3(mS20)
244
Perovskite#Ba2LaRuO6(I1-)
248
Perovskite#Ba2LaRuO6#Cu2NdNbO6
251
Perovskite#Ba2LaRuO6(I1-)
264
Perovskite#K2LiAlF6
269
Perovskite#Ba2LaRuO6#Cu2NdNbO6
279
Perovskite#

1392
Perovskite#Na3AlF6
1393
Perovskite#Na3AlF6
1394
Perovskite#Na3AlF6
1395
Perovskite#Na3AlF6
1396
Perovskite#Na3AlF6
1397
Perovskite#Na3AlF6
1398
Perovskite#Na3AlF6
1399
Perovskite#Na3AlF6
1400
Perovskite#Na3AlF6
1401
Perovskite#Na3AlF6
1402
Perovskite#Na3AlF6
1403
Perovskite#Na3AlF6
1404
Perovskite#Na3AlF6
1405
Perovskite#Na3AlF6
1406
Perovskite#Na3AlF6
1419
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1421
Perovskite#Na3AlF6
1422
Perovskite#Na3AlF6
1423
Perovskite#Na3AlF6
1425
Perovskite#BaBiO3(mS20)
1429
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1431
Perovskite#Na3AlF6
1434
Perovskite#K2LiAlF6
1440
Perovskite#(Nd,Ca)MnO3
1441
Perovskite#(Nd,Ca)MnO3
1442
Perovskite#BaBiO3(mS20)
1444
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1448
Perovskite#BaBiO3(mS20)
1450
Perovskite#BaBiO3(mS20)
1451
Perovskite#BaBiO3(mS20)
1458
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1459
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1460
Perovskite#Ba2LaRuO6#Cu2NdNbO6
1461
Perovskite_double#PrBaMn2O5+x
1462
Perovskite_double#PrBaMn2O5+x
1463
Perovskite_double#PrBaMn2O5

In [582]:
for l in range(len(dfoxide)):
    if dfoxide.Perovskite_label[l] == 1:
             print (dfoxide.SumFormula[l])

Co1 Mn1 O6 Y2
Ba1.4 O6 Pr1 Ru1 Sr0.6
Fe1 Ir1 O6 Sr2
Mg1 Nd2 O6 Ti1
Ir1 Lu2 Ni1 O6
Ca0.663 Nd0.702 O3 Sn0.635
Ca0.561 Nd0.864 O3 Sn0.575
Ba1 Bi0.5 O3 Sb0.5
Al1 Ca2 O6 Ta1
Fe1 Ir1 O6 Sr2
Ca1 Fe0.5 Nb0.5 O3
Na0.667 O3 Th0.334 Ti1
Co1.333 O11.997 Sb2.666 Sr3.999
Ca2 Mn1 O6 W1
Ba1 O3 Tb0.667 W0.333
Mn1 O5.8 Re1 Sr2
Co1 La1 O6 Ru1 Sr1
Ba2 Ir0.2 O6 Pr1 Ru0.8
O6 Ru1 Sr2 Tm1
Ir1 Na1 O6 Sm2
Ba0.96 Bi1 K0.04 O3
Ba1 Bi1 O3
Bi1 Nd1 O6 Sr2
Er1 O6 Ru1 Sr2
La1 Ni1 O6 Sb1 Sr1
La2 Li1 O6 Sb1
Ba2 Bi1.33 O6 Ru0.67
Ba2 La1 O6 Ru0.5 Sb0.5
Cu1 La1 O6 Sb1 Sr1
Ba1 Co1 Ir1 La1 O6
Ba2 Ir1 O6 Sr1
Ca2 Cr1 O6 Ta1
Co1 O6 Pb2 W1
Co1 La1 O6 Ru1 Sr1
Ba1 Co1 La1 O6 Ru1
Ca1 Ir1 O6 Sr2
Ir1 La2 Mg1 O6
Ir1 La2 O6 Zn1
Ir1 La2 Ni1 O6
Co1 Ir1 La2 O6
Ni1.338 O12 Sb2.662 Sr4
Ba2 Bi1 O6 Pr1
Ba2 Bi1 Nd1 O6
Ba2 Bi1 O6 Tb1
Ca2 Cr1 Nb1 O6
La0.597 Li0.21 O3 Ti1
La0.557 Li0.33 O3 Ti1
Ca2 Fe1 O6 Sb1
Fe1 O6 Sb1 Sr2
Bi0.267 O2.964 Pr0.733 Sr1
La2 Mg1 O6 Pt1
La2 Ni1 O6 Pt1
La2 O6 Pt1 Zn1
Co1 La2 O6 Pt1
Co1 O6 Pr2 Pt1
Co1 Nd2 O6 Pt1
Bi1 O3 

### Get rid of COMPLEX

In [591]:
aoxide = []

for l in range(len(dfoxide)):
    if '.' in dfoxide.StructuredFormula[l]:
        #print (df.StructuredFormula[l])
        aoxide.append(l)
        
dfoxide_complex = dfoxide.drop(a, axis=0)
dfoxide_complex.sort_values(by=['Perovskite_label'], ascending=False, inplace=True)
dfoxide_complex.reset_index(drop=True, inplace=True)
dfoxide_complex[186:189]

,CollectionCode,HMS,StructuredFormula,StructureType,Pressure,SumFormula,ChemicalName,Journal,PublicationYear,Perovskite_label
186,75596,P 1 21/n 1,La2 (Ir Zn O6),Perovskite#Ba2LaRuO6#Cu2NdNbO6,0.101325,Ir1 La2 O6 Zn1,Dilanthanum irido(IV)zincate,Materials Research Bulletin,1994,1
187,86298,P 1 21/n 1,Ca2 Er Nb O6,Perovskite#Na3AlF6,0.101325,Ca2 Er1 Nb1 O6,Dicalcium erbium niobium oxide,Zhurnal Neorganicheskoi Khimii,1981,1
188,86299,P 1 21/n 1,Ca2 Lu Nb O6,Perovskite#Na3AlF6,0.101325,Ca2 Lu1 Nb1 O6,Dicalcium lutetium niobium oxide,Zhurnal Neorganicheskoi Khimii,1981,1


### Get rid of DUPLICATE taking into account polymorphism

In [592]:
seenoxide = []
uniqoxide = []
for l in range(len(dfoxide_complex)):
    uniqoxide.append(l)
    if dfoxide_complex.StructuredFormula[l] not in seenoxide:
    #print(df_complex.StructuredFormula[l])
        del uniqoxide[-1]
        seenoxide.append(dfoxide_complex.StructuredFormula[l])

dfoxide_complex_multiplicate = dfoxide_complex.drop(uniqoxide, axis=0)
dfoxide_complex_multiplicate.reset_index(drop=True, inplace=True)
dfoxide_complex_multiplicate

,CollectionCode,HMS,StructuredFormula,StructureType,Pressure,SumFormula,ChemicalName,Journal,PublicationYear,Perovskite_label
0,246091,P 1 21/m 1,Ca2 (Mn Ta O6),"Perovskite#(Nd,Ca)MnO3",0.101325,Ca2 Mn1 O6 Ta1,Dicalcium mangano(III)tantalate(V),Journal of Solid State Chemistry,2008,1
1,153062,P 1 21/n 1,(Sr1.5 La0.5) (Mn W O6),Perovskite#Ba2LaRuO6#Cu2NdNbO6,0.101325,La0.5 Mn1 O6 Sr1.5 W1,Strontium lanthanum manganese tungsten oxide (...,Journal of Solid State Chemistry,2005,1
2,61434,P 1 21/n 1,Sr2 Sm Nb O6,Perovskite#Na3AlF6,0.101325,Nb1 O6 Sm1 Sr2,Distrontium samarium niobium oxide,Zhurnal Neorganicheskoi Khimii,1985,1
3,60777,P 1 21/n 1,Nd2 Mg Ti O6,Perovskite#Ba2LaRuO6#Cu2NdNbO6,0.101325,Mg1 Nd2 O6 Ti1,Dineodymium magnesium titanate(IV),"Acta Crystallographica, Section C: Crystal Str...",1986,1
4,153132,P 1 21/n 1,Sr2 Tb Ru O6,Perovskite#Na3AlF6,0.101325,O6 Ru1 Sr2 Tb1,Distrontium terbium ruthenium(V) oxide,Journal of Solid State Chemistry,2005,1
...,...,...,...,...,...,...,...,...,...,...
1931,167606,F m -3 m,Pb2 Sc (Ti0.5 Te0.5) O6,Ba2CaWO6,0.101325,O6 Pb2 Sc1 Te0.5 Ti0.5,Lead scandium titanium tellurium oxide (2/1/0....,Dalton Transactions,2010,0
1932,167504,F m -3 m,Ba2 Pb U O6,Elpasolite#K2NaAlF6#Ba2CaWO6,0.101325,Ba2 O6 Pb1 U1,Dibarium lead uranium hexaoxide,Radiokhimiya,2009,0
1933,167503,F m -3 m,Ba2 Cd U O6,Elpasolite#K2NaAlF6#Ba2CaWO6,0.101325,Ba2 Cd1 O6 U1,Dibarium cadmium uranium hexaoxide,Radiokhimiya,2009,0
1934,167502,F m -3 m,Ba2 Zn U O6,Elpasolite#K2NaAlF6#Ba2CaWO6,0.101325,Ba2 O6 U1 Zn1,Dibarium zinc uranium hexaoxide,Radiokhimiya,2009,0


## Code - FINISHED and now extraction

In [593]:
dfoxide_complex_multiplicate.to_csv('Parsed-oxides.csv', columns = ['StructuredFormula', 'Perovskite_label'], index=False)

# Second method using shell
Comment - I had issues in downloading the aiida framework and to get login/password for distant acces

## Using ssh to connect to the server hosting the database 

In [ ]:
!ssh -fN -L 3306:localhost:3306 -L 8010:localhost:80 aiida@theossrv2.epfl.ch > /dev/null 2>&1 #using username of someone from aiida team

## Loading the ICSD importer class and instantiate it

In [ ]:
importer_class = DbImporterFactory('icsd')
importer_parameters = {'server': 'http://theossrv2.epfl.ch',
            'host': '127.0.0.1',
            'db': 'icsd',
            'passwd': 'sql'

importer = importer_class(**importer_parameters)
importer

In [ ]:
importer_class = DbImporterFactory('icsd')

## Query the ICSD for a given database entry 



In [ ]:
icsd_id = '617290' #graphite

query_results = importer.query(id=icsd_id)

query_results_perovskyte_type = importer.query(spacegroup='F m -3 m')
# ou Fm3m avec une barre sur le 3 (Fm-3m??)

# Hall notation : -F 4 2 3
# Hermang : F m -3 m 
# number : 225


print len(query_results)
print len(query_results_perovskyte_type)

## Get the cif file corresponding to the entry, and convert it to an AiiDA structure



In [ ]:
entre = query_results.at(0)
cif = entry.get_cif_node()
print cif.get_formulae()

structure = cif._get_aiida_structure(converter='pymatgen')
print structure.get_formula()
print structure.get_ase()

## Get the spacegroup of the structure with spglib



In [ ]:
print spglib.get_spacegroup(structure.get_ase())

## Store the structure and prints its pk



In [ ]:
structure.store()
structure.pk

### Now access the terminal and type the following to visualise the structure


#### verdi data structure show --format xcrysden 5084



# MACHINE LEARNING


## Import multiple classification algorithms from scikt-learn


In [ ]:
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    from sklearn.naive_bayes import MultinomialNB
    model = MultinomialNB(alpha=0.01)
    model.fit(train_x, train_y)
    return model


# KNN Classifier
def knn_classifier(train_x, train_y):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(train_x, train_y)
    return model


# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model


# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=100)
    model.fit(train_x, train_y)
    return model


# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    from sklearn import tree
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)
    return model


# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    from sklearn.ensemble import GradientBoostingClassifier
    model = GradientBoostingClassifier(n_estimators=100)
    model.fit(train_x, train_y)
    return model


# SVM Classifier
def svm_classifier(train_x, train_y):
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', probability=True)
    model.fit(train_x, train_y)
    return model



 # 3 layer neural network classficiation
def mlp_classifier(train_x,train_y):
    from sklearn.neural_network import MLPClassifier
    model =  MLPClassifier(hidden_layer_sizes=(256,256,256), max_iter=200, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-6, random_state=1,
                    learning_rate_init=.1)
    model.fit(train_x,train_y)
    return model